# Data and Analysis Plan: Movie Reccomendation
## Team -23

- Vedant Bhagat (bhagat.ve@northeastern.edu)
- Anthony Lee (lee.ant@northeastern.edu)
- Dawn Lu (lu.daw@northeastern.edu)
- Josh Rosenberg (rosenberg.jo@northeastern.edu)


## Project Goal:
This work will scrape li sts f bomoviesrom [BoardGameGeek.com](https://boardgamegeek.com/) to reccmenmd a new bomovieo users who input a gamoviehey already enjoy. 

## Data 
<font color='green'>
(1%) Gives a summary of the data processing pipeline so a technical expert can easily follow along.
    
This overview section allows you to describe an overview of your data processing pipeline.  
    
Your pipeline should end by saving one or more csvs of data which will be loaded by the analysis portion of the notebook.
</font>

### Overview 
We will scrape a [list of all boardgames](https://boardgamegeek.com/browse/boardgame) ranked by popularity from BoardGameGeek.  

<img src="https://i.ibb.co/mD456p1/Screenshot-from-2021-03-14-09-25-01.png" width=800px>


From this list, for each game, we can obtain:
- title
- year published
- url to specific game page
- description (e.g. "Vanquish monsters ...")

Upon visiting an [individual game's webpage](https://boardgamegeek.com/boardgame/174430/gloomhaven) we can use the top titlebar:

<img src="https://i.ibb.co/6DdmKsg/Screenshot-from-2021-03-13-18-57-00.png" width=600px>

to observe:
- complexity rating
- playtime (mins a typical game lasts)
- min/max number of player required
- reccomended age range

Most importantly we seek to collect the category and mechanism tags associated with a particular game:
<img src="https://i.ibb.co/f1mbxVD/Screenshot-from-2021-03-13-18-56-52.png" width=150px>

Each game has multiple category tags.  Our expectation is that games with similar tags are enjoyed by similar players.  This tag data will be essential to reccomend a new game to a player who inputs another game they enjoy.  To simplify analysis, each tag will have its own column in our output DataFrame.  For example:

|       | card game | mystery | social | communication |
|-------|-----------|---------|--------|---------------|
| game0 | True      | True    | False  | False         |
| game1 | False     | True    | True   | True          |

Indicates that 
- game0 has tags 'card game' and 'mystery'
- game1 has tags ' mystery', 'social' and 'communication'

### Pipeline Overview

We will accomplish this task with three functions:
- `get_url()`
    - returns html string of a given url
- `clean_top_games()`
    - builds dataframe of [a single page of top games](https://boardgamegeek.com/browse/boardgame/page/2) from html string 
- `clean_game_meta()`
    - collects game meta data (tags, complexity, playtime, min/max players, age range ...) from an [individual game's webpage](https://boardgamegeek.com/boardgame/174430/gloomhaven)
    
As well as two short scripts:
- **Scrape list of games:** use `get_url()` and `clean_top_games()` in a loop to collect n pages of top games (100 * n games), populating a DataFrame `df_game`
- **Get meta data per game:** loop through each row of `df_game`, query and process the individual game's webpage via `get_url()` and `clean_game_meta()` using the previously collected url and append the remaining features to `df_game`

ggooooodegjsr;gnsw


### Pipeline
<font color='green'>
    
(4%) Obtains, cleans, and merges all data sources involved in the project.
    
Documentation counts!
    
The majority of this section is code, but do make sure that one can do a quick sanity check that your pipeline worked by printing a few examples (e.g. call `DataFrame.head()` a few times).
</font>


### Visualizations (sanity check / data exploration)
<font color='green'>
    
(2.5%) Builds two visualizations (graphs) from the data which characterize the distribution of the data itself in some interesting way. Your visualizations will be graded based on how much information they can effectively communicate with readers. Please make sure your visualizations are sufficiently distinct from each other.

**Wherever a non-technical reader may misunderstand, write a few sentences which specify how to interpret the graph**  It is expected that a non-technical reader can fully digest your graphs based on the images themselves as well as your explanatory text (they won't read your code).
    
Notice that the pipeline above takes 20 mins or so to complete.  We have manually re-labelled `game.csv` to `game_final.csv` so that:
- we don't overwrite our precious dataset
- we can load it below and continue on with our graphs

You won't lost any credit for not using a `_final.csv` structure in your code but I'd encourage you to do so to avoid losing your data / time :)
</font>



## Analysis Plan
<font color='green'>
    
(2%) Discuss what ML tools will be used and the relevant assumptions required to apply each. Either:

discuss why one algorithm may be chosen over the others

describe what subset of a whole suite of similar algorithms you’ll apply (its ok to say, ‘we’re going to try all of these because we don’t have reason to think one should be better than another’)

For example:

- Project goal: Predicting the movement of stock prices based on how similar companies are doing via regression.
    - This is observable: there are plenty of information you could collect about stock prices (though it will need some careful though about what a "similar" company is)
    - This is sound and simple: seems intuitive that there'd be a relationship to be found between the company's stock prices
    
- Project goal: Recommend a movie based on the events of a person's dream last night.
    - This is not observable: how will you get enough data on what people dreamed about?
    - This is not sound or simple: what kind of movies should we suggest for people with nightmares of falling?  Doesn't seem to be a clear relationship between dreams / movies a person would enjoy ....
    
    
To date, we've covered data collection and processing extensively and are only starting ML this week (Mar 22).  The grading will account for this:
- be specific / clear about which data will be used
    - give plenty of examples to illustrate things
- be as specific / clear as you can about the ML analysis
</font>

### Ovehttps://developers.themoviedb.org/3/movies/get-movie-details

In [2]:
import pandas as pd

movies = pd.read_csv('data/movies.csv')

movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
tags = pd.read_csv('data/tags.csv')

tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [4]:
ratings = pd.read_csv('data/ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [5]:
links = pd.read_csv('data/links.csv')

links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
import pandas as pd

movies = pd.read_csv('data/movies.csv')

movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
tags = pd.read_csv('data/tags.csv')

tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [8]:
ratings = pd.read_csv('data/ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [9]:
links = pd.read_csv('data/links.csv')

links.head()



,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### Fetching data is... hard

So, it turns out that making over 60,000 API requests sequentially is prone to failure. Lots of things can go wrong - wifi can drop, laptop can fall asleep, etc.

There are a couple of solutions to this:
  - Save api results locally as you get them, so you can resume where you left off if network drops
  - Run the code doing all the fetches in a cloud vm so that it has high network reliability and stability

We ended up doing both. I spun up a small debian vm in gcp, and ran the below cell as a standalone python script. Once the script finished running, I used SCP to copy the files from the cloud vm to my local machine, so that the data was available locally for my jupyter notebook to consume. 

If this were a business use case, and we needed to run this on a regular basis, I would set this up as a cron job that ran on a scheduled basis, and dumped the data to a new s3 bucket each time it ran


In [10]:
import requests
tmdb_api_key = '433618a0549fe010dec7ca3a9dc46d53'


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def build_data_cache_from_tmdb():
    """
    Fetch all movie data from TMDB, and store it in chunked json files
    We do this because network conditions are unreliable sometimes, maybe battery dies,
    etc, so we want to be able to resume where we left off, instead of losing all the data
    we gathered. Chunks are written as json files to the /cache directory.
    """
    tmdb_id_list = links['tmdbId'].tolist()
    chunked_list = list(chunks(tmdb_id_list, 100))

    for idx, chunk in enumerate(chunked_list):
        results = []
        for movie_id in chunk:
            url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US'
            resp = requests.get(url)
            data = resp.json()
            data['tmdb_id'] = movie_id
            results.append(data)
        movies_df = pd.DataFrame(results)
        start = idx * 100
        end = idx * 100 + len(chunk)
        movies_df.to_json(f'cache/movies_{start}_{end}_data.json')


In [11]:
# Usually I would use an environment variable to control this; However,
# Environment variables with Jupyter is tricky, so we'll just hardcode this
# boolean manually. You probably want to leave this as false and use the cache
# we've already built.
rebuild_cache = False
if rebuild_cache:
    build_data_cache_from_tmdb()



In [56]:


import os

directory = 'cache'
 
files = []
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      files.append(f)

dataframes = []

# Read each file, add it to the array
for file in files: 
  dataframes.append(pd.read_json(file))

# merge all dataframes into one
tmdb_movies_df = pd.concat(dataframes)
len(tmdb_movies_df)

62423

In [16]:
df_merged_with_gl_ids = pd.merge(tmdb_movies_df, links, how='inner', left_on = 'tmdb_id', right_on = 'tmdbId')

df_merged_with_gl_ids.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,video,vote_average,vote_count,tmdb_id,success,status_code,status_message,movieId,imdbId,tmdbId
0,0.0,None,None,0.0,"[{'id': 18, 'name': 'Drama'}]",,274104.0,tt0379176,es,B-Happy,...,0.0,5.3,6.0,274104.0,NaN,NaN,None,185857,379176,274104.0
1,0.0,/hSGMwIlns3hmuvd0VrnayJCdpZ8.jpg,None,10000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",https://www.facebook.com/pages/Rubens-Place-20...,256935.0,tt2609706,en,Ruben's Place,...,0.0,5.7,10.0,256935.0,NaN,NaN,None,185859,2609706,256935.0
2,0.0,/89JnVvAavvGfVP3KyTQZ9lzZs2M.jpg,None,0.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",,484997.0,tt6513338,cn,仙球大戰,...,0.0,0.0,0.0,484997.0,NaN,NaN,None,185861,6513338,484997.0
3,0.0,/nmdsYZvgqxwLkcgWCYCXQEcDxW.jpg,None,0.0,"[{'id': 10752, 'name': 'War'}, {'id': 18, 'nam...",,163363.0,tt0056160,hu,Két félidő a pokolban,...,0.0,7.7,15.0,163363.0,NaN,NaN,None,185865,56160,163363.0
4,0.0,/pyYMPkIFuvtB4CVVmBSrkYlTg5D.jpg,None,0.0,"[{'id': 99, 'name': 'Documentary'}]",None,445004.0,tt6440810,en,Iron Men,...,0.0,6.3,7.0,445004.0,NaN,NaN,None,185867,6440810,445004.0


In [89]:
df_averaged_gl_ratings = ratings.groupby('movieId')['rating'].mean().multiply(2).round().divide(2).reset_index()
df_gl_rating_count = ratings['movieId'].value_counts().rename_axis('movieId').reset_index(name='number_of_ratings')
df_gl_rating_count.head()


df_ratings_computed = pd.merge(df_averaged_gl_ratings, df_gl_rating_count, how='inner', on='movieId')
df_ratings_computed.head()


,movieId,rating,number_of_ratings
0,1,4.0,57309
1,2,3.5,24228
2,3,3.0,11804
3,4,3.0,2523
4,5,3.0,11714


In [90]:
df_merged_with_gl_ratings = pd.merge(df_merged_with_gl_ids, df_ratings_computed, how="inner", left_on = 'movieId', right_on = 'movieId')

We've done joins/merges on the numerical stuff, but what we really want is to convert some of these string attributes into usable things to train our model. The biggest one we want is genre. In order to get this into a more usable format, we will denormalize genre into many columns, i.e for each genre, we will add a column to the dataframe `is_<genre-name>`, for example, `is_fantasy`, `is_horror`, and so forth.

If a movie matches one of those genres, it will have a 1 in the corresponding column, otherwise it will have a 0

In [91]:

denormalized_genres = []

# movies.head()
for index, movie in movies.iterrows():
  # print(movie)
  genres = movie['genres'].split('|')
  movie_with_genres = { 'movieId': movie['movieId'] }
  for genre in genres: 
    movie_with_genres[f'is_genre_{genre.lower()}'] = 1
  denormalized_genres.append(movie_with_genres)

df_genres = pd.DataFrame(denormalized_genres)
df_genres.fillna(0, inplace=True)

df_merged_with_genres = pd.merge(df_merged_with_gl_ratings, df_genres, how='inner', left_on = 'movieId', right_on = 'movieId')
df_merged_with_genres.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,is_genre_horror,is_genre_mystery,is_genre_sci-fi,is_genre_imax,is_genre_documentary,is_genre_war,is_genre_musical,is_genre_western,is_genre_film-noir,is_genre_(no genres listed)
0,0.0,None,None,0.0,"[{'id': 18, 'name': 'Drama'}]",,274104.0,tt0379176,es,B-Happy,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,/hSGMwIlns3hmuvd0VrnayJCdpZ8.jpg,None,10000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",https://www.facebook.com/pages/Rubens-Place-20...,256935.0,tt2609706,en,Ruben's Place,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,/89JnVvAavvGfVP3KyTQZ9lzZs2M.jpg,None,0.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",,484997.0,tt6513338,cn,仙球大戰,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,/nmdsYZvgqxwLkcgWCYCXQEcDxW.jpg,None,0.0,"[{'id': 10752, 'name': 'War'}, {'id': 18, 'nam...",,163363.0,tt0056160,hu,Két félidő a pokolban,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,/pyYMPkIFuvtB4CVVmBSrkYlTg5D.jpg,None,0.0,"[{'id': 99, 'name': 'Documentary'}]",None,445004.0,tt6440810,en,Iron Men,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Future Improvements to our data
We would like to use the tags as well, but there is no good way to put them into the same dataframe as the rest of our data, because they're a seperate table and there will be 10000s of unique tags, so we can't denormalize them the same way we did with the genre. Same goes for denormalizing the people column and some of the other array columns in the main dataframe. Hopefully in class we will learn some more ways to train models on more free-text stuff like that

### Normalizing budget and viewership based on time
Due to inflation, population increases, in order to accurately use any data related to quantites that are affected by the flow of time (i.e budget, viewership, number of viewers), we need to find a way to normalize data based on the year that the movie was filmed. 